In [ ]:
import os
import torch
import numpy as np
import pandas as pd

from utils.configs import config
from utils.data_loader import get_test_loader, get_data_loaders
from utils.model_list import xception
from torchvision.models.feature_extraction import create_feature_extractor, get_graph_node_names

In [ ]:
device = torch.device('cpu')

In [ ]:
model_params = [config.fold]
MODEL_CKPT = os.path.abspath('model_weights/fold_{}/'.format(*model_params))

In [ ]:
net = xception(config=config)

In [ ]:
net.load_state_dict(torch.load(f'model_weights/fold_{config.fold}/best_f1.pth')['model'])

In [ ]:
net.to(device)
net.eval()

In [ ]:
train_dl, valid_dl = get_data_loaders(config, get_dataset=False, one_hot_labels=config.one_hot_labels)

In [ ]:
def get_correct_classified(y_true, y_hat):
    y_true = np.asarray(y_true)
    y_hat = np.argmax(y_hat, axis=1)
    res = y_hat == y_true
    return res

In [ ]:
logits_correct = []
label_correct = []
with torch.no_grad():
    for minibatch in valid_dl:
        x, y = minibatch
        out = net(x).detach().numpy()
        y = np.asarray(y)
        correct_index = get_correct_classified(y, out)

        out_correct = out[correct_index]
        y_out_correct = y[correct_index]

        for logits, label in zip(out_correct, y_out_correct):
            logits_correct.append(logits)
            label_correct.append(label)

logits_correct = np.asarray(logits_correct)
label_correct = np.asarray(label_correct)

In [ ]:
av_map = {}
for label in range(config.num_species.sum()):
    av_map[label] = logits_correct[label_correct == label]

In [ ]:
av_map